In [1]:
import lasio
import pathlib
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display
import os
pth = r'C:\Users\miahn\Documents\PyCode\Jupyter\LAS\9094449~XTO~Lawson 5-13H14X~20181018174046~FINAL_FILES_LAS_TIME~189912300000~196202021800~TM.las'

las = lasio.read(pth)

# Now we create the root name for all output files for use later on
originalPath = os.path.dirname(pth)
fName = os.path.basename(pth)
jobNumber = fName[:fName.find('~')]
operatorName = las.well['COMP'].value
wellName = las.well['WELL'].value
rootName = originalPath +'\\%s~%s~%s~DEPTH' % (jobNumber, operatorName, wellName)

'C:\\Users\\miahn\\Documents\\PyCode\\Jupyter\\LAS\\9094449~XTO Energy~Lawson 5-13H14X~DEPTH'

In [2]:
df = las.df()
# Drop columns and rows that only have null values
df.dropna(axis=1, how='all', inplace=True)
df.dropna(axis=0, how='all', inplace=True)

# Make TIME the index
df.index.names = ['TIME']

# Keep only drilling rows
df.drop(df[(df['RIGSTAT'] != 111) & (df['RIGSTAT'] != 11)].index, inplace=True)
''' There is an error in Cadence where it sometimes creates a Hole Depth in the millions. 
The bit depth however appears to be correct in these cases. To account for this we 
replace the value of Depth with Bit Depth. This should not cause an issue as we have 
already dropped non drilling rows, so Bit Depth and Hole Depth should be equal '''
df.DEPTH[df.DEPTH > 50000] = df.BITDEP

'''Just in case the above does not work, remove any row with a Hole Depth greater than 50k'''
df.drop(df[(df['DEPTH'] > 50000)].index, inplace=True)

'''Now we need to aggregate the data in 1 foot intervals. For this we chose to get the max
value for the aggregated data to see worst case scenerio for each foot.'''

# The first step is to make each depth a single foot. I am rounding the numbers here, but it might be better 
# to truncate or round up. We will have to experiment to figure that out.
df = df.round({'DEPTH': 0}) 

# After that we group the values by DEPTH
df = df.groupby(['DEPTH']).max()

'''LAS files require a uniform step value. WellCAD requires a uniform step value for well log data. The 
above can very easily leave depths out during areas with high rate of penetration. We need to fill in 
these missing depths. This will of course leave null values for everything except the depth int the new 
rows. We might consider interpolation later, however this will suffice to make the LAS file standard 
and WellCAD happy.'''
# Create a new index from the current index min and max, incrementing by 1. This will overwrite the 
# current index which was set to the DEPTH column when we grouped earlier, so we name the new index DEPTH
new_index = pd.Index(np.arange(df.index.min(),df.index.max(),1), name="DEPTH").astype(int) 

# Set the the index to our newly created index. Pandas will automatically expand all the available rows to this 
# new index based on the values in the old index

df = df.reindex(new_index)
# We reset the index here so that the new DEPTH index is moved back to the columns
df = df.reset_index()

# Output to csv just so we can compare to the LAS file later
df.to_csv(rootName + '.csv',  encoding='utf-8')

In [3]:
'''This is where things get a bit more complicated. We have the mnemonic names as our dataframe column headers, 
but the dataframe has no idea what the unit types are. Luckily the original LAS file does and the lasio 
library stores this information in the las file we originally read. 

Our goal is to compare the dataframes column names, extract only the items we want, change the values that need 
to be changed such as creation date, step value, presentation name, etc.'''

# Create a new LAS file and setup header
depthlas = lasio.LASFile()
depthlas.well.DATE = str(datetime.today())
depthlas.well.STRT = lasio.HeaderItem('STRT', 'ft', df.index.min(), 'START DEPTH')
depthlas.well.STOP = lasio.HeaderItem('STOP', 'ft', df.index.max(), 'STOP DEPTH')
depthlas.well.STEP = lasio.HeaderItem('STEP', 'ft', 1, 'STEP')
depthlas.well.NULL = lasio.HeaderItem('NULL', '', -999.25, 'Null Value')
depthlas.well.DATE = str(datetime.today())

# Add items from old header to new header
'''These are the options we want to ignore.'''
ignorelist = ['STRT', 'STOP', 'STEP', 'NULL', 'TITL', 'TIME', 'DATE']

# Add/Modify Well section
oldMnemonics =[]
for itm in las.well:
    if itm.original_mnemonic.find(':')==-1 and itm.original_mnemonic not in ignorelist:
        oldMnemonics.append(itm.original_mnemonic)
for listItem in oldMnemonics:
    depthlas.well[listItem] = las.well[listItem]

# Add/Modify paramters section
oldMnemonics =[]
for itm in las.params:
    if itm.original_mnemonic.find(':')==-1 and itm.original_mnemonic not in ignorelist:
        oldMnemonics.append(itm.original_mnemonic)
for listItem in oldMnemonics:
    depthlas.params[listItem] = las.params[listItem]

# Add data from dataframe
colList = df.columns
for col in colList:
    depthlas.add_curve(col, df[col], unit=las.curves[col].unit, descr=las.curves[col].descr[las.curves[col].descr.find(' '):]) 

# display(depthlas.header)
depthlas.write(rootName + '.las', version=2)
'''Congratulations! You should have depth based LAS and CSV files created from a Unix Time based LAS file'''

'Congratulations! You should have a depth based LAS file created from a Unix Time based LAS file'